<br>
<h1 style = "font-size:40px; font-family:cursive ; font-weight : normal; color : #7100B4; text-align: center; border-radius: 100px 100px;">⚡Catboost+XGBoost+Lightgbm+DNN with OPTUNA⚡</h1>
<br>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">✅Importing Required Libraries: </h1>

In [ ]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from catboost import CatBoostClassifier
from sklearn.feature_selection import chi2, f_classif, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
import lightgbm as lgbm
import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier
import optuna
import tqdm
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)


<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">🔍Basic Data Exploration: </h1>

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.nunique()[1:].plot.barh(figsize=(8,12))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.countplot(x='target', data=train)
ax.set_title('Target Distribution')


In [ ]:
train.drop(columns=['id']).describe().T.style.bar(subset=['mean'], color='#606ff2')\
                            .background_gradient(subset=['std'], cmap='magma_r')\
                            .background_gradient(subset=['50%'], cmap='summer')

In [ ]:
test.drop(columns=['id']).describe().T.style.bar(subset=['mean'], color='#606ff2')\
                            .background_gradient(subset=['std'], cmap='magma_r')\
                            .background_gradient(subset=['50%'], cmap='summer')

<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">⚒ Data Transformation </h1>

In [ ]:
le = LabelEncoder()
train['target'] = le.fit_transform(train['target'])
train.columns
cols = list(train.columns)
cols.remove("target")
cols.remove("id")

In [ ]:
not_features = ['id', 'target']
features = []
for feat in train.columns:
    if feat not in not_features:
        features.append(feat)
print(features)

In [ ]:
scaler = StandardScaler()
train[features] = scaler.fit_transform(train[features])
test[features] = scaler.transform(test[features])

In [ ]:
train

In [ ]:
X=train.drop(['target','id'],axis=1)
X


In [ ]:
Y=train['target']
Y

<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">🎯CatBoost with OPTUNA </h1>

In [ ]:
def objective(trial,data=X,target=Y):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3,random_state=42)
    params = {'iterations':trial.suggest_int("iterations", 4000, 15000),
              'od_wait':trial.suggest_int('od_wait', 500, 2300),
             'loss_function':'MultiClass',
              'task_type':"GPU",
              'eval_metric':'MultiClass',
              'leaf_estimation_method':'Newton',
              'bootstrap_type': 'Bernoulli',
              'learning_rate' : trial.suggest_uniform('learning_rate',0.02,0.3),
              'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
              'subsample': trial.suggest_uniform('subsample',0,1),
              'random_strength': trial.suggest_uniform('random_strength',10,30),
              'depth': trial.suggest_int('depth',1,6),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,6),
               }
    model = CatBoostClassifier(**params)  
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
        
    y_preds = model.predict_proba(X_test)


    log_loss_multi = log_loss(y_test, y_preds)
    
    return log_loss_multi

In [ ]:
OPTUNA_OPTIMIZATION = True

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)
print('Number of finished trials:', len(study.trials))
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
if OPTUNA_OPTIMIZATION:
    display(optuna.visualization.plot_optimization_history(study))
    display(optuna.visualization.plot_slice(study))
    #display(optuna.visualization.plot_intermediate_values(study))
    display(optuna.visualization.plot_parallel_coordinate(study))

In [ ]:
cat_params = study.best_trial.params
cat_params['loss_function'] = 'MultiClass'
cat_params['eval_metric'] = 'MultiClass'
cat_params['bootstrap_type']= 'Bernoulli'
cat_params['leaf_estimation_method'] = 'Newton'
cat_params['random_state'] = 42
cat_params['task_type']='GPU'
test_preds=None

kf = StratifiedKFold(n_splits = 10 , shuffle = True , random_state = 42)
for fold, (tr_index , val_index) in enumerate(kf.split(X.values , Y.values)):
    
    print("-" * 50)
    print(f"Fold {fold + 1}")
    
    x_train,x_val = X.values[tr_index] , X.values[val_index]
    y_train,y_val = Y.values[tr_index] , Y.values[val_index]
        
    eval_set = [(x_val, y_val)]
    
    model =CatBoostClassifier(**cat_params)
    model.fit(x_train, y_train, eval_set = eval_set, verbose = False)
    
    train_preds = model.predict(x_train)    
    val_preds = model.predict_proba(x_val)
    
    print(log_loss(y_val, val_preds))
    
    if test_preds is None:
        test_preds = model.predict_proba(test[cols].values)
    else:
        test_preds += model.predict_proba(test[cols].values)

print("-" * 50)
test_preds /= 10

In [ ]:
submission1 = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")
submission1['Class_1']=test_preds[:,0]
submission1['Class_2']=test_preds[:,1]
submission1['Class_3']=test_preds[:,2]
submission1['Class_4']=test_preds[:,3]
submission1.head()

<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">🎯XGBoost with OPTUNA </h1>

In [ ]:
def objective(trial,data=X,target=Y):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3,random_state=42)
    params = {
        "n_estimators": trial.suggest_int("n_estimators",200,2000,100),
        "subsample": trial.suggest_discrete_uniform("subsample",0.6,1,0.1),
        "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree",0.6,1,0.1),
        "eta": trial.suggest_loguniform("eta",1e-3,0.1),
        "reg_alpha": trial.suggest_int("reg_alpha",1,50),
        "reg_lambda": trial.suggest_int("reg_lambda",5,100),
        "max_depth": trial.suggest_int("max_depth",5,20),
        "min_child_weight": trial.suggest_int("min_child_weight",5,20),
    }
    model = xgb.XGBClassifier(**params, tree_method='gpu_hist', random_state=42)
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],verbose = False,eval_metric='mlogloss')
        
    y_preds = model.predict_proba(X_test)


    log_loss_multi = log_loss(y_test, y_preds)
    
    return log_loss_multi

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)
print('Number of finished trials:', len(study.trials))
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
if OPTUNA_OPTIMIZATION:
    display(optuna.visualization.plot_optimization_history(study))
    display(optuna.visualization.plot_slice(study))
    #display(optuna.visualization.plot_intermediate_values(study))
    display(optuna.visualization.plot_parallel_coordinate(study))

In [ ]:
xgb_params = study.best_trial.params
xgb_params['tree_method'] = 'gpu_hist'
xgb_params['random_state'] = 42
test_preds1=None

kf = StratifiedKFold(n_splits = 10 , shuffle = True , random_state = 42)
for fold, (tr_index , val_index) in enumerate(kf.split(X.values , Y.values)):
    
    print("-" * 50)
    print(f"Fold {fold + 1}")
    
    x_train,x_val = X.values[tr_index] , X.values[val_index]
    y_train,y_val = Y.values[tr_index] , Y.values[val_index]
        
    eval_set = [(x_val, y_val)]
    
    model1 = xgb.XGBClassifier(**xgb_params)
    model1.fit(x_train, y_train, eval_set = eval_set, eval_metric='mlogloss',verbose=False)
    
    train_preds = model1.predict(x_train)    
    val_preds = model1.predict_proba(x_val)
    
    print(log_loss(y_val, val_preds))
    
    if test_preds1 is None:
        test_preds1 = model1.predict_proba(test[cols].values)
    else:
        test_preds1 += model1.predict_proba(test[cols].values)

print("-" * 50)
test_preds1 /= 10

In [ ]:
submission2 = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")
submission2['Class_1']=test_preds1[:,0]
submission2['Class_2']=test_preds1[:,1]
submission2['Class_3']=test_preds1[:,2]
submission2['Class_4']=test_preds1[:,3]
submission2.head()

<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">🎯LightGBM with OPTUNA</h1>

In [ ]:
def objective(trial,data=X,target=Y):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3,random_state=42)
    params = {
        "objective": "multiclass",
        "num_class":4,
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    model = lgbm.LGBMClassifier(**params,random_state=42)
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],verbose = False)
        
    y_preds = model.predict_proba(X_test)


    log_loss_multi = log_loss(y_test, y_preds)
    
    return log_loss_multi

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)
print('Number of finished trials:', len(study.trials))
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
if OPTUNA_OPTIMIZATION:
    display(optuna.visualization.plot_optimization_history(study))
    display(optuna.visualization.plot_slice(study))
    #display(optuna.visualization.plot_intermediate_values(study))
    display(optuna.visualization.plot_parallel_coordinate(study))

In [ ]:
lgbm_params = study.best_trial.params
lgbm_params['objective'] = 'multiclass'
lgbm_params['random_state'] = 42
lgbm_params['metric'] = 'multi_logloss'
test_preds2=None

kf = StratifiedKFold(n_splits = 10 , shuffle = True , random_state = 42)
for fold, (tr_index , val_index) in enumerate(kf.split(X.values , Y.values)):
    
    print("-" * 50)
    print(f"Fold {fold + 1}")
    
    x_train,x_val = X.values[tr_index] , X.values[val_index]
    y_train,y_val = Y.values[tr_index] , Y.values[val_index]
        
    eval_set = [(x_val, y_val)]
    
    model2 = lgbm.LGBMClassifier(**lgbm_params)
    model2.fit(x_train, y_train, eval_set = eval_set,verbose=False)
    
    train_preds = model2.predict(x_train)    
    val_preds = model2.predict_proba(x_val)
    
    print(log_loss(y_val, val_preds))
    
    if test_preds2 is None:
        test_preds2 = model2.predict_proba(test[cols].values)
    else:
        test_preds2 += model2.predict_proba(test[cols].values)

print("-" * 50)
test_preds2 /= 10

In [ ]:
submission3 = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")
submission3['Class_1']=test_preds2[:,0]
submission3['Class_2']=test_preds2[:,1]
submission3['Class_3']=test_preds2[:,2]
submission3['Class_4']=test_preds2[:,3]
submission3.head()

In [ ]:
from keras import layers, models

<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">🎯DNN </h1>

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, input_shape=X.shape,input_dim=50,activation='relu', name="Hidden-1"))
layers.BatchNormalization()
model.add(layers.Dropout(0.2))

# model.add(layers.Dense(256, activation='relu'))
# layers.BatchNormalization()
# model.add(layers.Dropout(0.25))

# model.add(layers.Dense(128, activation='relu'))
# layers.BatchNormalization()
# model.add(layers.Dropout(0.25))

# model.add(layers.Dense(64, activation='relu'))
# layers.BatchNormalization()
# model.add(layers.Dropout(0.25))

# model.add(layers.Dense(32, activation='relu'))
# layers.BatchNormalization()
# model.add(layers.Dropout(0.25))

# model.add(layers.Dense(16, activation='relu'))
# layers.BatchNormalization()
# model.add(layers.Dropout(0.25))

model.add(layers.Dense(8, activation='relu'))
layers.BatchNormalization()
model.add(layers.Dropout(0.2))

model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
Y1 = pd.get_dummies(Y)
Y1

In [ ]:
history = model.fit(X, Y1, epochs=50,validation_split=0.3)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(history.history['loss'],lw=4)
plt.plot(history.history['val_loss'],lw=4)
plt.title('Model Loss',fontsize=20)
plt.ylabel('Loss',fontsize=15)
plt.xlabel('Epoch',fontsize=15)
plt.legend(['Train','Test'])
plt.show()

In [ ]:
test_preds3=model.predict_proba(test[cols].values)
submission4 = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")
submission4['Class_1']=test_preds3[:,0]
submission4['Class_2']=test_preds3[:,1]
submission4['Class_3']=test_preds3[:,2]
submission4['Class_4']=test_preds3[:,3]
submission4.head()

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.utils.plot_model(model, to_file="model.png", show_shapes=True)

In [ ]:
submission1.to_csv('sub1.csv',index=False)
submission2.to_csv('sub2.csv',index=False)
submission3.to_csv('sub3.csv',index=False)
submission4.to_csv('sub4.csv',index=False)

<h1 style = "font-family: cursive;display:fill; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #5CFBFA; color :#000000; border-radius: 150px 150px;">🌀Ensemble time </h1>

Playing with weights

In [ ]:
res1 = (2*submission1 + submission2 + submission3 + 2*submission4)/6
res1.to_csv("res1.csv",index=False)

In [ ]:
res2 = (submission1 + 2*submission2 + 2*submission3 + submission4)/6
res2.to_csv("res2.csv",index=False)

In [ ]:
res3 = (2*submission1 + 2*submission2 + submission3 + submission4)/6
res3.to_csv("res3.csv",index=False)

In [ ]:
res4 = (4*submission1 + 3*submission2 + 2*submission3 + submission4)/10
res4.to_csv("res4.csv",index=False)

![Upvote!](https://img.shields.io/badge/Upvote-If%20you%20like%20my%20work-07b3c8?style=for-the-badge&logo=kaggle)